# Day 1: PyTorch 数据加载基础（Dataset 与 DataLoader）

在深度学习项目中，数据的加载与预处理是整个流程的基础。本节重点介绍 PyTorch 提供的两个核心工具：
- **`torch.utils.data.Dataset`**：自定义数据集的抽象基类
- **`torch.utils.data.DataLoader`**：用于批量加载数据、支持多进程、自动 shuffle 等

通过这两个工具，我们可以快速构建高效、灵活的数据管道。

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [1]:
import os
import requests

save_dir = '../data/Adult'
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, 'X.csv')

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
if not os.path.exists(file_path):
    print(f"Downloading Adult dataset to {file_path}...")
    r = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(r.content)
    print("Download finished.")
else:
    print("Dataset already exists.")

Download finished.
Download finished.


## 自定义 Dataset 类：三个必需方法

自定义数据集类需要继承 `torch.utils.data.Dataset`，并重写以下**三个核心方法**：

1. **`__init__(self, ...)`**：初始化数据集
   - 加载数据文件（CSV、图像目录等）
   - 存储数据路径、标签、转换操作等
   
2. **`__len__(self)`**：返回数据集大小
   - 用于 DataLoader 计算 epoch 中的 batch 数量
   
3. **`__getitem__(self, idx)`**：根据索引返回数据项
   - 返回单个样本（通常是 (feature, label) 元组）
   - 在这里应用数据增强（augmentation）或转换

**设计原则**：Dataset 只负责返回单个样本，DataLoader 负责批量化与并行加载。

In [9]:
class MyDataset(Dataset):
    """
    自定义 Dataset 示例：从 CSV 文件加载数据
    
    Args:
        filepath (str): CSV 文件路径
        transform (callable, optional): 应用于样本的转换函数
    """
    def __init__(self, filepath='../data/Adult/X.csv', transform=None): 
        """加载数据并初始化"""
        self.data = pd.read_csv(filepath, header=None, na_values=' ?').select_dtypes(include=['number'])
        self.data = self.data.fillna(0)
        self.transform = transform

    def __len__(self):
        """返回数据集大小"""
        return len(self.data)

    def __getitem__(self, idx):
        """
        根据索引返回单个样本
        
        Args:
            idx (int): 样本索引
            
        Returns:
            样本（原始或转换后）
        """
        sample = self.data.iloc[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

In [10]:
class ToTensor:
    """
    将 pandas Series（一行数据）转换为 PyTorch tensor
    
    这个转换类可以在 Dataset 的 __getitem__ 中调用，用于数据预处理。
    """
    def __call__(self, sample):
        """
        Args:
            sample (pd.Series): 单行数据
            
        Returns:
            torch.Tensor: float32 类型的张量
        """
        return torch.tensor(sample.values, dtype=torch.float32)

In [11]:
# 创建 Dataset 实例并用 DataLoader 包装
data = MyDataset(transform=ToTensor())
dataloader = DataLoader(
    data, 
    batch_size=32, 
    shuffle=True,
    num_workers=0  # CPU 为 0，若要多进程则设置 > 0
)

print(f"数据集大小: {len(data)}")
print(f"DataLoader batch 数: {len(dataloader)}")

数据集大小: 32561
DataLoader batch 数: 1018


## 数据预处理与缺失值处理

**重要提示**：上面的 `MyDataset` 只读取了数值数据。实际项目中，**通常需要先做数据预处理**：

1. **处理缺失值**（pandas + sklearn）
2. **特征编码**（独热编码、标准化等）
3. **数据验证**（检查异常值、数据类型等）

**推荐流程**：

原始 CSV → pandas 清洗/预处理 → 保存为中间文件或缓存 → Dataset 读取 → Tensor 转换 → DataLoader 批量化


这样做的好处是预处理步骤只需运行一次，之后可以快速加载。

## 实践示例：迭代 DataLoader 并观察批量数据

In [12]:
batch_data = next(iter(dataloader))
print(f"一个批次数据形状: {batch_data.shape}") # 应为 [batch_size, num_features]

一个批次数据形状: torch.Size([32, 6])


## DataLoader 的关键参数

| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `batch_size` | 每批样本数 | 1 | 通常设为 32/64/128 |
| `shuffle` | 是否打乱数据 | False | 训练时应设为 True；验证/测试时设为 False |
| `num_workers` | 多进程数 | 0 | 0 表示单进程；>0 时可加速数据加载 |
| `pin_memory` | 是否锁定内存 | False | GPU 训练时可设为 True，加快数据传输 |
| `drop_last` | 是否丢弃最后不足一批的样本 | False | 某些情况（如 BatchNorm）下应设为 True |


## 常见问题与最佳实践

**Q1：什么时候使用 `num_workers > 0`？**
- 数据集很大（> 10K 样本）且 Dataset 加载较慢时，多进程可显著加速。
- 在 CPU 上开发时通常设为 0，避免多进程调试困难。

**Q2：为什么 DataLoader 迭代后内存没有释放？**
- 检查是否保持了对 batch 数据的引用。设为 `None` 显式释放或使用 `del`。

**Q3：如何在 Dataset 中实现数据增强（augmentation）？**
- 在 `__getitem__` 中应用转换：`if self.train: sample = apply_augmentation(sample)`

**Q4：Dataset 中可以返回多个输出吗（如 (feature, label, metadata)）？**
- 完全可以。DataLoader 会自动按位置堆叠，返回 `(batch_features, batch_labels, batch_metadata)`。

## 总结

Dataset 与 DataLoader 是 PyTorch 数据管道的核心：
- **Dataset**：定义数据加载与预处理逻辑（样本级别）
- **DataLoader**：自动进行批量化、打乱、多进程加载（批级别）

正确设计数据加载流程能显著提升训练效率与代码可维护性。